# Overview  <a name='objectives' />

This notebook introduces you to a few Unsupervised Learning methods.

The topics that will be covered are:

1. <a href=#k_means>Naive k-Means</a>
2. <a href=#fuzzy_k_means>Fuzzy k-Means</a>
3. <a href=#agglomerative_clustering>Agglomerative Clustering</a>

### Programming Tasks
For the programming tasks you will need to replace the following comment and exception with your own code:

```python
# YOUR CODE HERE
raise NotImplementedError()
```

Most programming tasks are followed by a cell with tests (using the `assert` function from python). You can use these cells while developing your implementation and for validating your implementation.

**<font size="3" color="red">Note</font>**: The `@contract` decorators make sure the data types and shapes are correct for the inputs and outputs. See [here](https://andreacensi.github.io/contracts/tour.html#quick-tour) for more. If you are more comfortable working without these, you can comment out the lines starting with `@contract`. However, in that case it can get tedious to locate the exact source of a bug.

### Open Questions
The notebook also contains a few open questions. For the open questions you can put your answer in the cell below the question.

In [0]:
# DO NOT INSTALL THE LIBRARIES WHEN WORKING ON ifi-europa.uibk.ac.at

# Make sure that the required libraries are installed
# If you are using Google Colab, remember to upload the requirements file before 
# running this cell
# If you are running this notebook locally, the requirements file needs to be in 
# the same location as this notebook
import os
running_local = True if os.getenv('JUPYTERHUB_USER') is None else False
    
if running_local:
    import sys
    !{sys.executable} -m pip install -r requirements_week08.txt

<a href=#objectives> [go to top] </a>
### Setup

In [0]:
%matplotlib notebook

from collections import namedtuple
import time
from IPython.display import set_matplotlib_formats

import matplotlib.pyplot as plt
import numpy as np
from scipy.cluster import hierarchy
from sklearn.datasets import make_blobs
from contracts import contract

set_matplotlib_formats('svg')

In [0]:
# Counter for figures
figcount = 0

# Set the random seed for reproducing results
random_seed = 3
np.random.seed(random_seed)

# Set colors
colors = plt.cm.tab10.colors

In [0]:
def plot_data(ax, X, labels=None, centroids=None):
    """Helper function for plotting data set and clustering result
    
    :param ax: Matplotlib Axes in which to plot the figure
    :param X: Dataset
    :param labels: Cluster labels for each data point in X
    :param centroids: Center of each cluster
    """
    
    # Plot all data points
    if labels is None:
        ax.scatter(X[:, 0], X[:, 1], 
                   s=30,
                   color=colors[0],
                   label=r'Data')
    # Plot data points with assigned cluster
    else:
        for label in range(min(labels), max(labels) + 1):
            ax.scatter(X[labels == label, 0], X[labels == label, 1], 
                   s=20,
                   color=colors[label],
                   alpha=0.5,
                   label=r'Cluster {}'.format(label))
            
    # Plot cluster means and contour
    if centroids is not None:
        n_clusters = centroids.shape[0]
        
        # Plot means
        for label in range(n_clusters):
            if label == 0:
                kwargs = {
                    'label': 'centroids'
                }
            else:
                kwargs = {}
            ax.scatter(centroids[label, 0], centroids[label, 1],
                       s=100,
                       marker='X',
                       color='black',
                       **kwargs)
            
        # Plot contours
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        
        # create grid to evaluate model
        xx = np.linspace(xlim[0], xlim[1], 100)
        yy = np.linspace(ylim[0], ylim[1], 100)
        YY, XX = np.meshgrid(yy, xx)
        xy = np.vstack([XX.ravel(), YY.ravel()]).T

        Z = assign_datapoints_to_cluster(xy, centroids)
        Z = np.asarray(Z).reshape(XX.shape)

        ax.contourf(XX, YY, Z, alpha=0.1, colors=colors, levels=n_clusters-1)
    
    # Set Labels and Limits
    ax.set_xlabel(r'$x_0$')
    ax.set_ylabel(r'$x_1$')
    ax.set_xlim(X[:, 0].min() - 0.1, X[:, 0].max() + 0.1)
    ax.set_ylim(X[:, 1].min() - 0.1, X[:, 1].max() + 0.1)
    
    # Legend
    pst = ax.legend(loc='upper left', frameon=True)
    pst.get_frame().set_edgecolor('k')

We will be using the `make_blobs` function from scikit-learn to generate a dataset. In this case we know the number of clusters, but in reality you often do not. Later on in the exercise, we want to see if our k-Means algorithm is able to find the optimal number of centers for clustering the data.

In [0]:
# Create dataset
blobs_centers = [[-1,-1], [1,-1], [0,1], [1, 1.5]]
blobs, _ = make_blobs(1000, centers=blobs_centers, cluster_std=0.5)

# Plot dateset
plt.figure()
ax = plt.gca()
plot_data(ax, blobs)

<a href=#objectives> [go to top] </a>
# Part 1: Naive k-Means <a name='k_means' />
The *Naive k-Means* (or just *k-Means*) algorithm is a commonly used clustering algorithm, one of the reasons for that is its simplicity. The *Naive k-Means* algorithm requires us to define the number of clusters $k$ beforehand. The algorithm consist of the following steps:

1. Initialize the center for each of the $k$ clusters (also called *centroids*).
2. Assign each datapoint in your dataset to the closest cluster.
3. Update the center of each cluster.
4. Calculate the sum of squared errors (SSE) for all the clusters (also known as *distortion*).
5. Evaluate whether the algorithm has converged (SSE has stabalised), if not return to step 2.

### Task 1.1: Initializing cluster centers
Choosing the right initial centers is important, if the wrong centers are chosen the algorithm might take very long to converge or not converge at all. An easy but effective way for initializing the centers is by sampling $k$ random datapoint from you dataset and assign them as initial cluster centers.

In [0]:
@contract(X='array[AxB], A>0, B>0', 
          k='int, >0', 
          returns='array[CxB], C>0, B>0')
def init_cluster_centers(X, k):
    """Initialize centers of clusters
    
    Initialize the center of each cluster
    by selecting a random data point from X. 
    
    Each row in the return centers will represent a
    cluster center and the row index represents the 
    label of that cluster. For example:
    
      centroids = np.array([[1, 2],
                            [4, 5]])
                           
      - Cluster 0 is represented by the first row and its
        center is np.array([1, 2])
      - Cluster 1 is represented by the second row and
        its center is np.array([4, 5])
    
    
    :param X: Dataset (numpy array)
    :param k: Number of clusters (int)
    :param centers: The centers of the clusters (numpy array)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return centers

In [0]:
# Test with random data
X = np.random.uniform(-10, 10, size=(50, 4))
k = 3

assert init_cluster_centers(X, k).shape == (k, X.shape[1])

# Test different number of clusters
k = 4
assert init_cluster_centers(blobs, k).shape == (k, blobs.shape[1])


# Test on blobs dataset
assert init_cluster_centers(blobs, k).shape == (k, blobs.shape[1])

### Task 1.2: Computing distances to cluster
Next we want to compte the distance of each datapoint in $\mathbf{X}$ to each cluster center using the *squared euclidean distance*. For two datapoint $\mathbf{x} = [x_0,\ x_1,\ \ldots,\ x_M]$ and $\mathbf{y} = [y_0,\ y_1,\ \ldots,\ y_M]$ the squared euclidean distance is defined as:

$$d(\mathbf{x}, \mathbf{y}) = \|\mathbf{x} - \mathbf{y}\|^2 = (\mathbf{x} - \mathbf{y})^T (\mathbf{x} - \mathbf{y}) = \sum_{i=0}^M(x_i - y_i)^2$$

In [0]:
@contract(X='array[NxM], N>0, M>0',
          center='array[M] | array[1xM], M>0',
          returns='array[N], N>0')
def compute_distance_to_cluster(X, center):
    """Calculate squared euclidean distance between datapoint in X and a cluster
    
    :param X: Dataset (numpy array)
    :param center: Center of one cluster (numpy array)
    :returns: distance of each datapoint to the centroid (numpy array)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return dists

In [0]:
# Test on some random data
X = np.ones((5,2))
center = np.zeros((1,2))
dists = compute_distance_to_cluster(X, center)

assert dists.shape == (X.shape[0],)
assert np.allclose(dists, np.ones(X.shape[0]) * 2.0)


# Test on blobs dataset
assert compute_distance_to_cluster(blobs, blobs[0,:]).shape == (blobs.shape[0],)

### Task 1.3: Assign datapoint to cluster
Assign each datapoint in $\mathbf{X}$ to a cluster based on the shortest distance between the datapoint and the cluster center. Use the `compute_distance_to_cluster` function defined above.

In [0]:
@contract(X='array[NxM], N>0, M>0',
          centers='array[KxM], K>0, M>0',
          returns='array[N], N>0')
def assign_datapoints_to_cluster(X, centers):
    """Assign each datapoint in X to a cluster
    
    Remember that each row in centers represents a cluster
    and the label of that cluster is the row index.
    
    :param X: Dataset (numpy array)
    :param centers: Centers of the clusters (numpy array)
    :returns: labels of assigned cluster (numpy array)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return labels

In [0]:
# Test on some random data
X = np.vstack([np.ones((5,2)), np.zeros((5,2))])
centers = np.array([[1, 1], [0, 0]])
labels = assign_datapoints_to_cluster(X, centers)

assert labels.shape == (X.shape[0],)
assert np.allclose(labels[:5], 0)
assert np.allclose(labels[-5:], 1)


# Test on blobs dataset
k = 3
centers = init_cluster_centers(blobs, k)
labels = assign_datapoints_to_cluster(blobs, centers)
assert labels.shape == (blobs.shape[0],)

### Task 1.4: Compute new cluster centers
Next we can compute a new center of each cluster by taking the mean of all the datapoints in that cluster.

$$\mathbf{y}_k = \frac{1}{|X_k|}\sum_{\mathbf{x} \in X_k}\mathbf{x}$$

where $\mathbf{y}_k$ is the new center for cluster $k$, $X_k$ is the set of datapoints in cluster $k$ and $|X_k|$ is the number of datapoints in cluster $k$.

In [0]:
@contract(X='array[NxM], N>0, M>0',
          labels='array[N], N>0',
          centers='array[KxM], K>0, M>0',
          returns='array[KxM], K>0, M>0')
def update_centers(X, labels, centers):
    """Compute new cluster centers
    
    Note: It might be the case that a cluster has not been assigned,
          i.e., |X_k| = 0. This means we cannot compute a new center
          for this cluster.
          
          Think of a solution to this problem and implement it.
    
    :param X: Dataset (numpy array)
    :param labels: Cluster label of each datapoint in X (numpy array)
    :param centers: Cluster centers (numpy array)
    :param k: Number of clusters (int)
    :returns: New cluster centers (numpy array)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return new_centers

In [0]:
# Test on some random data
X = np.vstack([np.ones((5,2)), np.zeros((5,2))])
centers = np.array([[2.0, 2.0], [-1.0, -1.0]])
labels = assign_datapoints_to_cluster(X, centers)

new_centers = update_centers(X, labels, centers)
assert new_centers.shape == centers.shape
assert np.allclose(new_centers, np.array([[1., 1.], [0., 0.]]))

# Test with an empty cluster
X = np.random.uniform(-5., 5., size=(10, 2))
centers = np.array([[1, 2], [-1, 2], [0.4, 4]])
labels = np.random.choice([0, 1], size=(10,)) # Note that cluster label 2 will not occur in labels

new_centers = update_centers(X, labels, centers)
assert not np.isnan(new_centers).any()


# Test on blobs datadataset
k = 3
centers = init_cluster_centers(blobs, k)
labels = assign_datapoints_to_cluster(blobs, centers)

new_centers = update_centers(blobs, labels, centers)
assert new_centers.shape == centers.shape
assert not np.allclose(new_centers, centers)

### Task 1.5: Computing the Sum of Squared Errors (SSE)
The *distortion* is measured as the sum of the distance between each datapoint and its closest cluster center, i.e., the sum of squared errors (SSE):

$$\text{SSE} = \sum_{i=0}^K\sum_{\mathbf{x}\in X_i}\|\mathbf{x} - \mathbf{y}_i\|^2$$

where $K$ is the number of clusters, $X_i$ is the set of datapoints in cluster $i$ and $y_i$ is the center of cluster $i$.

In [0]:
@contract(X='array[NxM], N>0, M>0',
          labels='array[N], N>0',
          centers='array[KxM], K>0, M>0',
          returns='float, >=0')
def compute_sse(X, labels, centers):
    """Compute Sum of Squared Errors
    
    Note: You can use your compute_distance_to_cluster function.
    
    Note: It might be the case that a cluster has not been assigned,
          i.e., |X_i| = 0. This means we cannot compute a new center
          for this cluster.
          
          Think of a solution to this problem and implement it.
    
    :param X: Dataset (numpy array)
    :param labels: Cluster label of each datapoint in X (numpy array)
    :param centers: Cluster centers (numpy array)
    :param returns: SSE (float)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
        
    return sse

In [0]:
# Test on some random data
k = 2
X = np.vstack([np.ones((5,2)), np.zeros((5,2))])
centers = np.array([[2.0, 2.0], [-1.0, -1.0]])
labels = assign_datapoints_to_cluster(X, centers)

sse = compute_sse(X, labels, centers)
assert sse >= 0
assert abs(sse - 20.0) < 1e-5

centers = np.array([[1.0, 1.0], [0.0, 0.0]])
labels = assign_datapoints_to_cluster(X, centers)

sse = compute_sse(X, labels, centers)
assert abs(sse) < 1e-5


# Test on blobs dataset
k = 3
centers = init_cluster_centers(blobs, k)
labels = assign_datapoints_to_cluster(blobs, centers)

sse = compute_sse(blobs, labels, centers)
assert sse >= 0

### Task 1.6: Determine whether the algorithm has converged
k-Means is an iterative algorithm and we need a way to determine when to stop. There are two approaches, the first one is running the algorithm for a pre-defined number of iterations and use the final result. The second approach is stopping when the algorithm has converged. For naive k-Means this means stopping when the SSE does not change anymore between iterations (more precisely, changes less than some threshold).

Often a combination of both approaches is used, without the limiting the maximum number of iteration the algorithm could hypothetically go on forever.

In [0]:
@contract(prev_sse='float, >=0',
          cur_sse='float, >=0',
          threshold='float, >0')
def is_converged(prev_sse, cur_sse, threshold):
    """Check whether the algorithm has converged
    
    The algorithm has converged when the difference
    in SSE is smaller then some threshold.
    
    :param prev_sse: SSE of previous iteration (float)
    :param cur_sse: SSE of current iteration (float)
    :param threshold: Threshold for comparing SSE's (float)
    :param returns: True if converged, False otherwise (bool)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
        
    return converged

In [0]:
# Test on some random data
threshold = 1e-5
k = 2
X = np.vstack([np.ones((5,2)), np.zeros((5,2))])
centers = np.array([[2.0, 2.0], [-1.0, -1.0]])
labels = assign_datapoints_to_cluster(X, centers)
prev_sse = compute_sse(X, labels, centers)
centers = update_centers(X, labels, centers)
cur_sse = compute_sse(X, labels, centers)

assert is_converged(prev_sse, cur_sse, threshold) == False
assert is_converged(cur_sse, cur_sse, threshold) == True


# Test on blobs dataset
k = 3
centers = init_cluster_centers(blobs, k)
labels = assign_datapoints_to_cluster(blobs, centers)
prev_sse = compute_sse(blobs, labels, centers)
centers = update_centers(blobs, labels, centers)
cur_sse = compute_sse(blobs, labels, centers)

assert is_converged(prev_sse, cur_sse, threshold) == False

### Task 1.7: Combining all the parts
Now it is time to combine all the parts together. We have supplied a helper class for visualizing the progress of k-Means as an animation, this is a nice way of seeing how the clusters change each iteration. After that you will find the cell with the `k-means` function, which you have to implement.

In [0]:
class PlotKMeans:
    """Helper class for visualizing k-Means
    
    This class can be used to visualize the progress of our k-Means algorithm.
    Each iteration the figure can be updated with the new results.
    
    The parameter `sleep_time` can be used to increase or decrease the speed
    of the animation. If you put it to low you will not be able to see the changes
    but the learning will be doen faster.
    """
    
    def __init__(self, title, sleep_time=0.1):
        """
        :param title: Title to show above the figure (str)
        :param sleep_time: Amount of time to sleep between each iteration (float)
        """
        self.sleep_time = sleep_time        
        self.itrs = []
        self.sses = []
        
        self.fig, self.axes = plt.subplots(1, 2, figsize=(9.5, 4))
        
        self.fig.suptitle(title)
        self.fig.subplots_adjust(wspace=0.3)
        
    def update(self, X, labels, cluster_centers, itr, sse):        
        # Get left and right axes
        ax_l, ax_r = self.axes
        ax_l.clear()
        ax_r.clear()
        
        # Plot clusters in left figure using our orignal plot_data function
        plot_data(ax_l, X, labels, cluster_centers)
        
        # Store current iteration and sse
        self.itrs.append(itr)
        self.sses.append(sse)
        
        # Plot SSE in right figure
        if len(self.itrs) > 1:
            ax_r.plot(self.itrs, self.sses)

            # Set Labels and Limits
            ax_r.set_xlabel('Iteration')
            ax_r.set_ylabel('Sum of Squared Error (SSE)')
            ax_r.set_xlim(min(self.itrs), max(self.itrs))
            ax_r.set_ylim(min(self.sses) - 2, max(self.sses) + 2)
                        
        # Redraw plot
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()

        # Sleep for a little while
        time.sleep(self.sleep_time)           

In [0]:
@contract(k='int, >0',
          X='array[NxM], N>0, M>0',
          max_itr='int, >0',
          threshold='float, >0')
def k_means(k, X, max_itr=1000, threshold=1e-2, plot=None):
    """Naive k-Means algorithm
    
    You have to implement the k-Means algorithm using the functions
    defined above. Remember that the k-Means algorithm consists of
    the following steps:
     
      1. Initialize cluster centers
      2. Assign each datapoint in X to a cluster
      3. Update the cluster centers
      4. Compute the SSE (distortion)
      5. Stop if the algorithm has converged or reached the 
         maximum number of iterations. Otherwise go to Step 2.
    
    
    Some additional notes:
    
      - A helper class for visualizing the progress of the k-Means
        algorithm is provided and passed as the `plot` argument into
        this function. After step 4 you can update the figure using
        the following lines:
        
          # Plot iteration results
          if plot is not None:
              plot.update(X, labels, centers, itr, sse)
              
        where `X` is the dataset, `labels`are labels assigned to each datapoint
        in X based on the current cluster centers, `centers` are the 
        current cluster centers, `itr` is the current iteration and 
        `sse` is the current Sum of Squared Errors.
        
      - You can create a loop that goes from `0` to `max_itr` and use
        a `break` statement when the convergence criteria is met, i.e.,
        the `is_converged` function returns `True`.
        
      - When the algorithm stops at step 5, the current value of labels was
        determined using the cluster centers of the previous iteration. Re-assign
        the cluster labels using the latest values for the cluster centers.
    
    
    :param k: Number of clusters (int)
    :param X: Dataset (numpy array)
    :param max_itr: Maximum number of iterations (int)
    :param threshold: Stopping threshold for change in SSE (float)
    :param plot: PlotKMeans instance for visualizing results or None 
                 if you don't want to visualize the progress (PlotKMeans or None)
    :returns: A Python tuple containing:
                :param centers: Cluster centers
                :param labels: The labels for each datapoint in X calculated with the latest cluster
                               centers
                :param itr: Final iteration
                :param sse: Final SSE
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return centers, labels, sse, itr

In [0]:
k = 3
plot = PlotKMeans(title="{} clusters".format(k), sleep_time=0.25)
centers, labels, sse, itr = k_means(k, blobs, max_itr=100, plot=plot)

print("\nAlgorithm converged in {} iterations!\n".format(itr))
print("Final SSE: {}".format(sse))
print("Cluster means:")
for label in range(k):
    print("  Cluster {}: {}".format(label, centers[label, :]))

### Task 1.8: Choosing number of clusters using SSE
Usually you do not know beforehand what the ideal number of clusters is for you dataset. One way to determine the number of clusters is by comparing the final SSE for different number of clusters.

In [0]:
range_k = range(1, 20)
sses = []

for k in range_k:
    _, _, sse, itr = k_means(k, blobs, max_itr=100, plot=None)
    print("n_clusters = {}, converged in {} iterations, sse = {:.2f}".format(k, itr, sse))
    sses.append(sse)

In [0]:
fig = plt.figure()
ax = fig.gca()
ax.plot(range_k, sses)
ax.set_xlabel("Number of clusters")
ax.set_ylabel("Sum Squared Errors")
ax.set_xlim(range_k[0], range_k[-1])
_ = ax.set_ylim(0, max(sses))

As you can see, the SSE declines in an exponential way for a greater number of clusters. It is important to note that the SSE will keep decreasing and will eventually go to zero when the number of clusters equals the number of datapoints.

Think about why that is and when looking at the Figure what do you think would be a good number of clusters? Give your answer in the cell below.

In [0]:
# Assign the number of clusters to variable k:
#   k =
#
# The next cell will use this variable to solve the k-means problem

# YOUR CODE HERE
raise NotImplementedError()

In [0]:
plot = PlotKMeans(title="{} clusters".format(k), sleep_time=0.25)
centers, labels, sse, itr = k_means(k, blobs, max_itr=100, plot=plot)

print("\nAlgorithm converged in {} iterations!\n".format(itr))
print("Final SSE: {}".format(sse))
print("Cluster means:")
for label in range(k):
    print("  Cluster {}: {}".format(label, centers[label, :]))

We know that our blob data set was generated using 4 centers. There is a big change this also the value you choose above, if not run the above cell with `k=4`. 

Now it is interesting to compare the centers found by our k-means algorithm with the true centers. The cell below prints the true centers. They centers are probably in a different sequence but you can match each true center pretty well with a center found by the k-Means algorithm.

In [0]:
for center in blobs_centers:
    print(center)

<a href=#objectives> [go to top] </a>
# Part 2: Fuzzy k-Means <a name='fuzzy_k_means' />
In the second part of this assignment you will be implementing Fuzzy k-Means. Some of the components of the k-Means algorithm you implemented above can be reused, like the `init_cluster_centers` function.

### Task 2.1: Computing the weight matrix
In Fuzzy k-Means a single datapoint can belong to multiple clusters. The degree of membership of a datapoint $\mathbf{x}_i$ to cluster $k$ is given by the probability $P(w_k | \mathbf{x}_i)$. The probability is calculated using the following equation:

$$P(w_k | \mathbf{x}_i) = \frac{1}{\sum_{j=0}^K\left(\frac{\|\mathbf{x}_i - \mathbf{y}_k\|}{\|\mathbf{x}_i - \mathbf{y}_j\|}\right)^{\frac{2}{m - 1}}},$$

where $K$ is the total number of clusters, $y_i$ is the center of cluster $i$ and $m$ is the blending parameter (See slide 25 of the VO for more information).

The first task is to compute the weight matrix:

$$w = \left[\begin{array}{cccc}
    P(w_0 | \mathbf{x}_0) & P(w_1 | \mathbf{x}_0) & \cdots & P(w_K | \mathbf{x}_0) \\
    P(w_0 | \mathbf{x}_1) & P(w_1 | \mathbf{x}_1) & \cdots & P(w_K | \mathbf{x}_1) \\
    \vdots & \vdots & \vdots & \vdots \\
    P(w_0 | \mathbf{x}_N) & P(w_1 | \mathbf{x}_N) & \cdots & P(w_K | \mathbf{x}_N) \\
\end{array}\right],$$

where $K$ is the number of clusters and $N$ the number of datapoints in our dataset.

In [0]:
@contract(X='array[NxM], N>0, M>0',
          centers='array[KxM], K>0, M>0',
          m='float | int, > 1')
def compute_weight_matrix(X, centers, m):
    """Compute the weight matrix w as defined above
    
    Note: keep in mind that division by zero can occur.
          Implement appropriate measure to prevent this
          from happening.
          
    Note: the weight matrix w constists of conditional 
          probabilities, this means that the sum of each
          row has to equal one. (Why equals the sum of each
          row one and not each column?)
          
    :param X: Dataset (numpy array)
    :param centers: Cluster centers (numpy array)
    :param m: Blending parameters (float)
    :returns: Weight matrix w (numpy array)
    """
    assert m > 1
    
        
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return w

In [0]:
# Test on some random data
m = 2
X = np.random.uniform(-5., 5., size=(10, 2))
centers = np.array([[1, 2], [-1, 2], [0.4, 4]])

w = compute_weight_matrix(X, centers, m)

assert w.shape == (X.shape[0], centers.shape[0])
assert np.allclose(np.sum(w, axis=1), 1.)

# Test on blobs datadataset
k = 3
centers = init_cluster_centers(blobs, k)

w = compute_weight_matrix(blobs, centers, m)

assert w.shape == (blobs.shape[0], k)
assert np.allclose(np.sum(w, axis=1), 1.)

### Task 2.2: Compute cluster centers
Compute the cluster centers using the weight matrix according to:

$$\mathbf{y}_k = \frac{\sum_{i=0}^{N}P(w_k | \mathbf{x}_i)^m\mathbf{x}_i}{\sum_{i=0}^NP(w_k | \mathbf{x}_i)^m}$$

In [0]:
@contract(X='array[NxM], N>0, M>0',
          w='array[NxK], N>0, K>0',
          m='float | int, >1')
def compute_centers_fuzzy(X, w, m):
    """Compute centers for fuzzy k-means
    
    :param X: Dataset (numpy array)
    :param w: Weight matrix (numpy array)
    :param m: Blending paramter (float)
    :returns: Cluster centers (numpy array)
    """
    # YOUR CODE HERE
    raise NotImplementedError()

    return centers

In [0]:
# Test on some random data
m = 2
X = np.random.uniform(-5., 5., size=(10, 2))
centers = np.array([[1, 2], [-1, 2], [0.4, 4]])
w = compute_weight_matrix(X, centers, m)

new_centers = compute_centers_fuzzy(X, w, m)

assert new_centers.shape == centers.shape

# Test on blobs datadataset
k = 3
centers = init_cluster_centers(blobs, k)
w = compute_weight_matrix(blobs, centers, m)

new_centers = compute_centers_fuzzy(blobs, w, m)

assert new_centers.shape == centers.shape

### Task 2.3: Determine if the algorithm has converged
In naive k-Means we compared the SSE between two iteration to determine convergence. For fuzzy k-Means we compare the maximum weight value between two iterations:

$$|\underset{i, k}{\max}P^{t}(w_k | \mathbf{x}_i) - \underset{i, k}{\max}P^{t+1}(w_k | \mathbf{x}_i)| < \epsilon,$$

where $|\cdot|$ denotes the absolute value, superscript $t$ and $t+1$ denote subsequent iterations and $\epsilon$ some threshold value.

In [0]:
@contract(prev_w='array[NxK], N>0, K>0',
          new_w='array[NxK], N>0, K>0',
          threshold='float, > 0')
def is_converged_fuzzy(prev_w, new_w, threshold):
    """Determine whether fuzzy k-Means has converged
    
    :param prev_w: Weight matrix of previous iteration (numpy array)
    :param new_w: Weight matrix of current iteration (numpy array)
    :param threshold: Threshold for comparing maximum weight values (float)
    :param returns: True if converged, False otherwise (bool)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return converged

In [0]:
# Test on some random data
threshold = 1e-5
m = 2
k = 3
X = np.random.uniform(-5., 5., size=(10, 2))
centers = init_cluster_centers(X, k)
w = compute_weight_matrix(X, centers, m)
centers = compute_centers_fuzzy(X, w, m)
new_w = compute_weight_matrix(X, centers, m)

assert is_converged_fuzzy(w, new_w, threshold) == False
assert is_converged_fuzzy(new_w, new_w, threshold) == True


# Test on blobs dataset
k = 3
centers = init_cluster_centers(blobs, k)
w = compute_weight_matrix(blobs, centers, m)
centers = compute_centers_fuzzy(blobs, w, m)
new_w = compute_weight_matrix(blobs, centers, m)

assert is_converged_fuzzy(w, new_w, threshold) == False

### Task 2.4: Assigning cluster label
In the end we often still want to assign a datapoint to a single cluster. In fuzzy k-Means we assign a datapoint to the cluster for which the membership probability is the highest:

$$\text{label}_i = \underset{k}{\arg \max}P(w_k | \mathbf{x_i}),$$

where $\text{label}_i$ is the label for datapoint $\mathbf{x}_i$.

In [0]:
@contract(w='array[NxK], N>0, K>0',
          returns='array[N], N>0')
def assign_datapoints_to_cluster_fuzzy(w):
    """Assign cluster labels to each datapoint
    
    Note: notice that we do not need to use dataset
          X because each row in w represents a datapoint
          in our dataset X.
    
    :param w: Weight matrix (numpy array)
    :returns: Labels of assigned cluster (numpy array)
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return labels

In [0]:
# Test on some random data
threshold = 1e-5
m = 2
k = 3
X = np.random.uniform(-5., 5., size=(10, 2))
centers = init_cluster_centers(X, k)
w = compute_weight_matrix(X, centers, m)

labels = assign_datapoints_to_cluster_fuzzy(w)

assert labels.shape == (X.shape[0],)

# Test on blobs dataset
k = 3
centers = init_cluster_centers(blobs, k)
w = compute_weight_matrix(blobs, centers, m)

labels = assign_datapoints_to_cluster_fuzzy(w)

assert labels.shape == (blobs.shape[0],)

### Task 2.5: Combining all the parts
We can now put all the parts together. The sum of squared errors (SSE) was the objective function used by the naive k-means algorithm and therefore had an integral part in the algorithm. But it is also used as metric for evaluating the quality of a trained clustering algorithm. We will use it again for evaluating and tracking the progress of our fuzzy k-Means clustering algorithm.

In your implementation of fuzzy k-Means you can use the previous implemention `compute_sse` together with the `assign_datapoint_to_cluster_fuzzy` function for calculating the SSE.

In [0]:
@contract(k='int, >0',
          m='float | int, >1',
          X='array[NxM], N>0, M>0',
          max_itr='int, >0',
          threshold='float, >0')
def fuzzy_k_means(k, m, X, max_itr=1000, threshold=1e-5, plot=None):
    """Fuzzy k-Means algorithm
    
    You have to implement the fuzzy k-Means algorithm using the functions
    defined above and some functions from the naive k-Means algorithm. 
    
    The fuzzy k-Means algorithm consists of the following steps:
     
      1. Initialize cluster centers (use `init_cluster_centers` function)
         and initialize weight matrix w
      2. Update cluster centers
      3. Update weight matrix w
      4. Stop if the algorithm has converged or reached the 
         maximum number of iterations. Otherwise go to Step 2.
       
       
    Additional steps to implement:
      
      - Use the original `compute_sse` function to compute the sum of
        squared errors (SSE) in each iteration. In order to use this 
        function you need to compute the cluster labels first.
    
    
    Some additional notes:
    
      - A helper class for visualizing the progress of the k-Means
        algorithm is provided and passed as the `plot` argument into
        this function. After step 4 you can update the figure using
        the following lines:
        
          # Plot iteration results
          if plot is not None:
              plot.update(X, labels, centers, itr, sse)
              
        where `X` is the dataset, `labels`are labels assigned to each datapoint
        in X based on the current cluster centers, `centers` are the 
        current cluster centers, `itr` is the current iteration and 
        `sse` is the current Sum of Squared Errors.
        
      - You can create a loop that goes from `0` to `max_itr` and use
        a `break` statement when the convergence criteria is met, i.e.,
        the `is_converged` function returns `True`.
    
    
    :param k: Number of clusters (int)
    :param X: Dataset (numpy array)
    :param m: Blending parameter (float)
    :param max_itr: Maximum number of iterations (int)
    :param threshold: Stopping threshold for change in SSE (float)
    :param plot: PlotKMeans instance for visualizing results or None 
                 if you don't want to visualize the progress (PlotKMeans or None)
    :returns: A Python tuple containing:
                :param centers: Cluster centers
                :param labels: The labels for each datapoint in X calculated with the latest cluster
                               centers
                :param itr: Final iteration
                :param sse: Final SSE
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return centers, w, sse, itr

In [0]:
k = 3
m = 2
plot = PlotKMeans(title="{} clusters".format(k), sleep_time=0.25)
centers, w, sse, itr = fuzzy_k_means(k, m, blobs, max_itr=100, plot=plot)

print("\nAlgorithm converged in {} iterations!\n".format(itr))
print("Final SSE: {}".format(sse))
print("Cluster means:")
for label in range(k):
    print("  Cluster {}: {}".format(label, centers[label, :]))

### Task 2.6: Influence of $k$ and $m$
The fuzzy k-means algorithm has an extra parameter $m$, called the blending parameter. The question is, how do we choose $m$? For the naive k-means algorithm we compared the SSE for different number of clusters to determine the ideal number of clusters. 

We can take a similar approach for determining $m$, we will calculate the final SSE for different combinations of $m$ and the number of clusters. The code below will has a double looping over different values of $m$ and number of clusters $k$. It may take a few minutes to compute all the SSE values.

In [0]:
k_range = range(2, 9, 2)
m_range = [1.1, 1.5, 2., 5., 10.]
sses = []
for n_clusters in k_range:
    sse_cs = []
    for m in m_range:
        _, _, sse, itr = fuzzy_k_means(n_clusters, m, blobs, max_itr=50)
        print("n_clusters = {}, m = {}, converged in {} iterations, sse = {:.2f}".format(n_clusters, m, itr, sse))
        sse_cs.append(sse)
    sses.append(sse_cs)
sses = np.asarray(sses)

The next cell will again plot the number of clusters versus the SSE, each line represents a value of $m$

In [0]:
fig = plt.figure()
ax = fig.gca()
ax.plot(k_range, sses)
ax.set_xlabel("Number of clusters")
ax.set_ylabel("Sum Squared Errors")
ax.set_xlim(k_range[0], k_range[-1])
ax.set_ylim(0, np.max(sses))
ax.legend([r"m = {}".format(m) for m in m_range])

### Question
Observing the figure above, which value of for $m$ do you think is optimal. Choose from the evaluated set $\{1.1, 1.5, 2.0, 5.0, 10.0\}$. 

In [0]:
# Assign to variable m_opt:
#   m_opt = 

# YOUR CODE HERE
raise NotImplementedError()

In [0]:
# This cell contains hidden tests which are not visible for Students. 
# You don't have to do anything with it.


<a href=#objectives> [go to top] </a>
# Part 3: Agglomerative Clustering <a name='agglomerative_clustering'>
In the final part we will have a look at agglomerative clustering. Agglomerative is a hierarchical clustering approach. Meaning it creates a hierarchical structure in the data, this means the algorithm allows you to extract anywhere from $k=1$ to $k=N$ clusters from the same representation.

The agglomerative algorithm consists of the following 4 steps:

1. Start with $N$ singleton clusters
2. Find nearest clusters
3. Merge them
4. If $K > 1$, go to step 2, otherwise stop

We have already implemented most of the agglomerative algorithm for you, given in de cells below. Your task will be to implement two linkage functions (*single* and *complete* linkage). The linkage function determines the distance between to clusters. 

**Single linkage** calculates the smallest distance between two datapoints of two clusters:

$$\underset{\mathbf{x} \in \mathbf{X}_{k=i}, \mathbf{x}' \in \mathbf{X}_{k=j}}{\min}\| \mathbf{x} - \mathbf{x}' \|,$$

where $\mathbf{X}_{k=i}$ are datapoints in cluster $k=i$, $\mathbf{X}_{k=j}$ are datapoints in cluster $k=j$ and $\|\cdot\|$ is the euclidean distance.

**Complete linkage** calculates the largest distance between two datapoints of two clusters:

$$\underset{\mathbf{x} \in \mathbf{X}_{k=i}, \mathbf{x}' \in \mathbf{X}_{k=j}}{\max}\| \mathbf{x} - \mathbf{x}' \|,$$

where $\mathbf{X}_{k=i}$ are datapoints in cluster $k=i$, $\mathbf{X}_{k=j}$ are datapoints in cluster $k=j$ and $\|\cdot\|$ is the euclidean distance.

In [0]:
@contract(X='array[NxM], N>0, M>0',
          returns='array[N-1x4], N>1')
def agglomerative_clustering(X, linkage_fn):
    """Agglomerative clustering algorithm
    
    We use the scipy convention for storing the hierarchical
    clustering results in a linkage matrix Z. This allows us
    to use the dendogram plotting function from scipy for 
    visualizing the results. For more information on the 
    linkage matrix see:
    
      https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html
   
    :param X: Dataset (numpy array)
    :param linkage_fn: Function returning distance between two clusters (function)
    :returns: Linkage matrix Z
    """
    # Initialize empty linkage matrix
    Z = []
    
    # Step 1: Keep track of cluster indices
    clusters_idx = [idx for idx in range(X.shape[0])]
    
    # Keep track of datapoints within each cluster
    clusters_points = [[x] for x in X]
       
    # Cluster index tracker
    next_idx = len(clusters_idx)
    
    # Step 4: Loop until only one cluster is left
    while len(clusters_idx) > 1:
        # Step 2: Find the two cluster to merge and their distance
        i, j, dist = linkage(clusters_points, linkage_fn)
        
        # Step 3: Merge the original clusters into a new clusters
        new_cluster_points = clusters_points[i] + clusters_points[j]
                
        # Store new cluster index and points in the new cluster
        clusters_idx.append(next_idx)
        clusters_points.append(new_cluster_points)
        
        # Store merge in linkage matrix
        Z.append([clusters_idx[i], clusters_idx[j], dist, len(new_cluster_points)])
        
        # Delete merged clusters from clusters_idx and clusters_points
        # After merging clusters we cannot merge them again, therefore
        # we remove them from the list. The information about this merging
        # is stored in linkage matrix Z
        for idx in sorted([i, j], reverse=True):
            del clusters_idx[idx]
            del clusters_points[idx]
            
        # Increment cluster index
        next_idx += 1
        
    return np.asarray(Z)

def linkage(clusters, linkage_fn):
    """Find the two clusters that are closest to eachother
    
    The distance between two clusters is determined by the linkage function.
    
    """
    n_clusters = len(clusters)
    min_dist = np.inf
    idx = (-1, -1)
    
    for i in range(n_clusters):
        for j in range(i+1, n_clusters):
            dist = linkage_fn(np.asarray(clusters[i]), np.asarray(clusters[j]))
            if dist < min_dist:
                idx = (i, j)
                min_dist = dist
            
    return idx[0], idx[1], min_dist

### Task 3.1: Linkage functions
Define a function that computes the single linkage distance and one that computes the complete linkage between two clusters.

In [0]:
@contract(cluster_1='array[AxM], A>0, M>0',
          cluster_2='array[BxM], B>0, M>0')
def single(cluster_1, cluster_2):
    """Compute single linkage distance between cluster_1 and cluster_2
    
    :param cluster_1: (numpy array)
    :param cluster_2: (numpy array)
    :returns: Minimum distance between cluster_1 and cluster_2
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return dist

@contract(cluster_1='array[AxM], A>0, M>0',
          cluster_2='array[BxM], B>0, M>0')
def complete(cluster_1, cluster_2):
    """Compute complete linkage distance between cluster_1 and cluster_2
    
    :param cluster_1: (numpy array)
    :param cluster_2: (numpy array)
    :returns: Maximum distance between cluster_1 and cluster_2
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return dist

In [0]:
cluster_1 = np.asarray([[1], [8]])
cluster_2 = np.asarray([[2], [3], [4]])

assert single(cluster_1, cluster_2) < complete(cluster_1, cluster_2)
assert np.allclose(single(cluster_1, cluster_2), 1.0)
assert np.allclose(single(cluster_2, cluster_1), 1.0) # Ordering of clusters should not matter
assert np.allclose(complete(cluster_1, cluster_2), 6.0)
assert np.allclose(complete(cluster_2, cluster_1), 6.0)

### Task 3.2: Dendogram
Next we will compare the different linkage functions on a simple toy dataset and plot each dendogram. We will use the scipy function `scipy.hierarchy.dendogram` for plotting. See the VO lecture slide 34 and the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) for more information.

In [0]:
X = np.array([[i] for i in [2, 8, 0, 4, 1, 9, 9, 0]])

Z_single = agglomerative_clustering(X, single)
Z_complete = agglomerative_clustering(X, complete)

fig, axes = plt.subplots(1, 2, sharey=True)
hierarchy.dendrogram(Z_single, ax=axes[0])
hierarchy.dendrogram(Z_complete, ax=axes[1])

axes[0].set_ylabel("Distance")
axes[0].set_title("Single Linkage")
axes[1].set_title("Complete Linkage")

print("single linkage:\n{}".format(Z_single))
print("")
print("complete linkage:\n{}".format(Z_complete))

### Scipy implementation
Below we perform the same agglomerative clustering using Scipy. The results should be the same as above. Note that the linkage matrices might be different because the merging sequence can be different when more then two clusters have the same distance to each other. The plots should show the same results

In [0]:
X = np.array([[i] for i in [2, 8, 0, 4, 1, 9, 9, 0]])
                            
Z_single_scipy = hierarchy.linkage(X, 'single')
Z_complete_scipy = hierarchy.linkage(X, 'complete')

fig, axes = plt.subplots(1, 2, sharey=True)
hierarchy.dendrogram(Z_single_scipy, ax=axes[0])
hierarchy.dendrogram(Z_complete_scipy, ax=axes[1])

axes[0].set_ylabel("Distance")
axes[0].set_title("Single Linkage")
axes[1].set_title("Complete Linkage")

print("single linkage:\n{}".format(Z_single_scipy))
print("")
print("complete linkage:\n{}".format(Z_complete_scipy))